[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://drive.google.com/file/d/1cTuUip7jpIWAxE5rXVv3M1quIab742rP/view?usp=sharing)

# CRISPR screen analysis with Perturb-tools

In this tutorial, we will cover
*  Loading three .csv files each about guide, sample, and guide count information into single Screen object
*  Slicing (indexing) Screen object to subset/select guides and samples
*  Adding Screen object to combine technical replicates
*  Concatenating Screen object to combine biological replicates
*  Normalize and calculate log fold change of guides across two different conditions
*  Writing Screen object to .h5ad or .xlsx file

In [1]:
import pandas as pd
import anndata as ad
import perturb_tools as pt

We will download public CRISPR/Cas9 Knock-out dataset: [TKO](http://tko.ccbr.utoronto.ca/) HeLa data.

In [2]:
! wget http://tko.ccbr.utoronto.ca/Data/readcount-HeLa-lib1.gz -nc
! wget http://tko.ccbr.utoronto.ca/Data/readcount-HeLa-lib2.gz -nc
! gunzip readcount-HeLa-lib1.gz -f
! gunzip readcount-HeLa-lib2.gz -f

--2023-04-10 23:47:18--  http://tko.ccbr.utoronto.ca/Data/readcount-HeLa-lib1.gz
Resolving tko.ccbr.utoronto.ca (tko.ccbr.utoronto.ca)... 142.150.76.126
Connecting to tko.ccbr.utoronto.ca (tko.ccbr.utoronto.ca)|142.150.76.126|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2877516 (2.7M) [application/x-gzip]
Saving to: ‘readcount-HeLa-lib1.gz’

100%[======================================>] 2,877,516   14.8MB/s   in 0.2s   

2023-04-10 23:47:18 (14.8 MB/s) - ‘readcount-HeLa-lib1.gz’ saved [2877516/2877516]

--2023-04-10 23:47:18--  http://tko.ccbr.utoronto.ca/Data/readcount-HeLa-lib2.gz
Resolving tko.ccbr.utoronto.ca (tko.ccbr.utoronto.ca)... 142.150.76.126
Connecting to tko.ccbr.utoronto.ca (tko.ccbr.utoronto.ca)|142.150.76.126|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2680113 (2.6M) [application/x-gzip]
Saving to: ‘readcount-HeLa-lib2.gz’

100%[======================================>] 2,680,113   15.7MB/s   in 0.2s   

2023-04-10

In [3]:
!cut -f 1,2 readcount-HeLa-lib1 > guide_info-HeLa-lib1.tsv
!cut -f 1,3- readcount-HeLa-lib1 > guide_count-HeLa-lib1.tsv
!cut -f 1,2 readcount-HeLa-lib2 > guide_info-HeLa-lib2.tsv
!cut -f 1,3- readcount-HeLa-lib2 > guide_count-HeLa-lib2.tsv

In [34]:
! head guide_info-HeLa-lib1.tsv

GENE_CLONE	GENE
A1BG_CACCTTCGAGCTGCTGCGCG	A1BG
A1BG_AAGAGCGCCTCGGTCCCAGC	A1BG
A1BG_TGGACTTCCAGCTACGGCGC	A1BG
A1BG_CACTGGCGCCATCGAGAGCC	A1BG
A1BG_GCTCGGGCTTGTCCACAGGA	A1BG
A1BG_CAAGAGAAAGACCACGAGCA	A1BG
A1CF_CGTGGCTATTTGGCATACAC	A1CF
A1CF_GGTATACTCTCCTTGCAGCA	A1CF
A1CF_GACATGGTATTGCAGTAGAC	A1CF


# Loading text file to AnnData object

Basic structure of AnnData object contains 3 types of information:
*  `Screen.X`: guide count matrix (numpy array with shape (n_guides, n_samples)
*  `Screen.var`: **guide RNA** information ex) sequence, target element (pandas DataFrame with length n_guides)
*  `Screen.obs`: **sample** information that gave rise to the guide counts (pandas DataFrame with length n_samples)

You can construct Screen object using any number of these three elements.

In [4]:
screen = pt.io.read_csv(X_path="guide_count-HeLa-lib1.tsv", guide_path="guide_info-HeLa-lib1.tsv", sample_path=None, sep="\t")

/data/pinello/PROJECTS/2023_03_scCRISPR_PIPELINE/perturb-tools/perturb_tools/_readwrite/_funcs/_to_from_csv.py:54: FutureWarning: X.dtype being converted to np.float32 from int64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  return ad.AnnData(X=X, var=guide_df, obs=sample_df)
/data/pinello/SHARED_SOFTWARE/anaconda_latest/envs/igvf_sccrispr/lib/python3.10/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [5]:
screen.X

array([[310.,  46., 239., ..., 508.,  97.,  91.],
       [226.,   1., 216., ..., 479.,  50., 115.],
       [338.,   0., 285., ..., 248.,  10., 130.],
       ...,
       [ 49.,   1., 250., ..., 331.,  28.,  76.],
       [296.,  52., 269., ..., 386.,  23., 120.],
       [469., 213., 363., ..., 566.,   3., 390.]], dtype=float32)

Alternatively, you can manually read the file and initialize Screen object.

In [6]:
tbl = pd.read_csv("readcount-HeLa-lib1", sep = "\t")
tbl2 = pd.read_csv("readcount-HeLa-lib2", sep = "\t")

In [29]:
tbl.iloc[:,:2]

,GENE_CLONE,GENE
0,A1BG_CACCTTCGAGCTGCTGCGCG,A1BG
1,A1BG_AAGAGCGCCTCGGTCCCAGC,A1BG
2,A1BG_TGGACTTCCAGCTACGGCGC,A1BG
3,A1BG_CACTGGCGCCATCGAGAGCC,A1BG
4,A1BG_GCTCGGGCTTGTCCACAGGA,A1BG
...,...,...
91315,luciferase_CCTCTAGAGGATGGAACCGC,luciferase
91316,luciferase_ACAACTTTACCGACCGCGCC,luciferase
91317,luciferase_CTTGTCGTATCCCTGGAAGA,luciferase
91318,luciferase_GGCTATGAAGAGATACGCCC,luciferase


In [30]:
def make_anndata(tbl):
  sample_df = pd.DataFrame(tbl.columns[2:]).rename(columns={0:"sample"}).set_index("sample")
  sample_df["replicate"] = sample_df.index.str[-1]
  sample_df["time"] = sample_df.index.str[1:-1].map(lambda s: int(s) if s else -1)
  return ad.AnnData(X=tbl.values[:,2:].T, dtype=int, var=tbl.iloc[:,:2].set_index("GENE_CLONE"), 
                   obs=sample_df)

In [31]:
adata = make_anndata(tbl)
bdata = make_anndata(tbl2)

In [32]:
adata.obs

,replicate,time
sample,,
T08A,A,8
T08B,B,8
T08C,C,8
T12A,A,12
T12B,B,12
T12C,C,12
T15A,A,15
T15B,B,15
T15C,C,15


In [33]:
adata.var

,GENE
GENE_CLONE,
A1BG_CACCTTCGAGCTGCTGCGCG,A1BG
A1BG_AAGAGCGCCTCGGTCCCAGC,A1BG
A1BG_TGGACTTCCAGCTACGGCGC,A1BG
A1BG_CACTGGCGCCATCGAGAGCC,A1BG
A1BG_GCTCGGGCTTGTCCACAGGA,A1BG
...,...
luciferase_CCTCTAGAGGATGGAACCGC,luciferase
luciferase_ACAACTTTACCGACCGCGCC,luciferase
luciferase_CTTGTCGTATCCCTGGAAGA,luciferase


### Slicing

In [35]:
adata_cut = adata[:, adata.var.GENE == "A1BG"]
adata_cut

View of AnnData object with n_obs × n_vars = 13 × 6
    obs: 'replicate', 'time'
    var: 'GENE'

In [36]:
adata_t8 = adata[adata.obs.time == 8, :]
adata_t8

View of AnnData object with n_obs × n_vars = 3 × 91320
    obs: 'replicate', 'time'
    var: 'GENE'

### Writing

In [37]:
adata.write("HeLa_lib1.h5ad")

## Arithmetic

### Adding
If the guide and conditions are exactly the same, objects can be added (ex. technical replicates).

In [38]:
pt.add(adata, adata)

/data/pinello/PROJECTS/2023_03_scCRISPR_PIPELINE/perturb-tools/perturb_tools/_framework/_screen.py:48: FutureWarning: X.dtype being converted to np.float32 from int64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  return AnnData(


AnnData object with n_obs × n_vars = 13 × 91320
    obs: 'replicate', 'time'
    var: 'GENE'

### Concatenating
Biological replicates can be concatenated along axes.

In [39]:
ad.concat((adata, adata), axis=0)

/data/pinello/SHARED_SOFTWARE/anaconda_latest/envs/igvf_sccrispr/lib/python3.10/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


AnnData object with n_obs × n_vars = 26 × 91320
    obs: 'replicate', 'time'

## Normalization & LFC calculation

In [40]:
pt.pp.log_norm(adata)

In [41]:
adata.layers['lognorm_counts']

array([[3.83616552, 1.57091719, 3.4906079 , ..., 4.50880773, 2.366281  ,
        2.29249441],
       [3.72344557, 0.07590494, 3.66320179, ..., 4.74827466, 1.88795355,
        2.85050474],
       [4.36367384, 0.        , 4.13058593, ..., 3.94220047, 0.65946991,
        3.09314233],
       ...,
       [2.09998439, 0.09367186, 4.15142457, ..., 4.53632282, 1.5252234 ,
        2.60840185],
       [4.01236793, 1.87158158, 3.88332224, ..., 4.37437528, 1.1218406 ,
        2.8353031 ],
       [3.57526448, 2.57512434, 3.24056381, ..., 3.82558522, 0.09740613,
        3.33346213]])

Calculating the LFC between T=18 vs T=8

In [42]:
lfcs = pt.pp.log_fold_change(adata, "T18A", "T08A", return_result=True)

Result can be saved to `.var`.

In [43]:
pt.pp.log_fold_change(adata, "T18A", "T08A")
adata.var

,GENE,T18A_T08A.lfc
GENE_CLONE,,
A1BG_CACCTTCGAGCTGCTGCGCG,A1BG,-0.341628
A1BG_AAGAGCGCCTCGGTCCCAGC,A1BG,-1.570917
A1BG_TGGACTTCCAGCTACGGCGC,A1BG,-0.060598
A1BG_CACTGGCGCCATCGAGAGCC,A1BG,-0.279654
A1BG_GCTCGGGCTTGTCCACAGGA,A1BG,0.912812
...,...,...
luciferase_CCTCTAGAGGATGGAACCGC,luciferase,0.946178
luciferase_ACAACTTTACCGACCGCGCC,luciferase,0.848203
luciferase_CTTGTCGTATCCCTGGAAGA,luciferase,0.026328


Calculating the T=18 vs T=8 across all replicates

In [51]:
adata_t = adata[adata.obs.replicate != "0", :].copy()

In [52]:
pt.pp.log_fold_change_reps(adata_t, 18, 8, rep_col="replicate", compare_col="time")

,A.18_8.lfc,B.18_8.lfc,C.18_8.lfc
GENE_CLONE,,,
A1BG_CACCTTCGAGCTGCTGCGCG,-0.341628,-1.623461,-0.351306
A1BG_AAGAGCGCCTCGGTCCCAGC,-1.570917,0.017767,1.871582
A1BG_TGGACTTCCAGCTACGGCGC,-0.060598,0.488223,-0.247264
A1BG_CACTGGCGCCATCGAGAGCC,-0.279654,0.893945,0.169561
A1BG_GCTCGGGCTTGTCCACAGGA,0.912812,1.240359,0.766479
...,...,...,...
luciferase_CCTCTAGAGGATGGAACCGC,0.946178,0.096590,0.677018
luciferase_ACAACTTTACCGACCGCGCC,0.848203,2.029437,0.480288
luciferase_CTTGTCGTATCCCTGGAAGA,0.026328,-0.211952,0.432175


Aggregate the LFCs based on `aggregate_fn [median, mean, sd]`.

In [53]:
pt.pp.log_fold_change_aggregate(adata_t, 8, 18, aggregate_col="replicate", compare_col="time", aggregate_fn = "median")

In [54]:
adata_t.var

,GENE,T18A_T08A.lfc,8_18.lfc.median
GENE_CLONE,,,
A1BG_CACCTTCGAGCTGCTGCGCG,A1BG,-0.341628,0.351306
A1BG_AAGAGCGCCTCGGTCCCAGC,A1BG,-1.570917,-0.017767
A1BG_TGGACTTCCAGCTACGGCGC,A1BG,-0.060598,0.060598
A1BG_CACTGGCGCCATCGAGAGCC,A1BG,-0.279654,-0.169561
A1BG_GCTCGGGCTTGTCCACAGGA,A1BG,0.912812,-0.912812
...,...,...,...
luciferase_CCTCTAGAGGATGGAACCGC,luciferase,0.946178,-0.677018
luciferase_ACAACTTTACCGACCGCGCC,luciferase,0.848203,-0.848203
luciferase_CTTGTCGTATCCCTGGAAGA,luciferase,0.026328,-0.026328


# Writing

In [48]:
pt.io.to_Excel(adata[:,:100], "Hela_lib_sub.xlsx")

Writing to: Hela_lib_sub.xlsx

	Sheet 1:	X
	Sheet 2:	lognorm_counts
	Sheet 3:	guides
	Sheet 4:	samples


In [49]:
pt.io.to_mageck_input(adata, "Hela_mageck_input.txt", target_column="GENE")

In [50]:
! head Hela_mageck_input.txt

sgRNA	gene	T08A	T08B	T08C	T12A	T12B	T12C	T15A	T15B	T15C	T18A	T18B	T18C	T0
A1BG_CACCTTCGAGCTGCTGCGCG	A1BG	310	226	338	356	249	224	186	60	296	125	49	296	469
A1BG_AAGAGCGCCTCGGTCCCAGC	A1BG	46	1	0	7	22	142	0	1	52	0	1	52	213
A1BG_TGGACTTCCAGCTACGGCGC	A1BG	239	216	285	117	244	116	172	298	269	119	250	269	363
A1BG_CACTGGCGCCATCGAGAGCC	A1BG	289	83	166	164	111	14	184	160	214	122	137	214	678
A1BG_GCTCGGGCTTGTCCACAGGA	A1BG	205	34	217	205	148	355	326	100	432	212	85	432	559
A1BG_CAAGAGAAAGACCACGAGCA	A1BG	389	331	468	1074	364	158	664	286	499	464	235	499	647
A1CF_CGTGGCTATTTGGCATACAC	A1CF	452	240	390	630	509	261	471	255	301	322	210	301	898
A1CF_GGTATACTCTCCTTGCAGCA	A1CF	71	30	29	119	155	153	131	76	56	94	61	56	199
A1CF_GACATGGTATTGCAGTAGAC	A1CF	207	227	223	118	141	173	176	198	42	118	166	42	271
